In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from src.agent_logic import Agent
import pandas
import numpy

## Init Agent

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
model = nn.Sequential(
    nn.Linear(16, 16),
    nn.Hardsigmoid(),
    nn.Linear(16, 8),
    nn.Hardsigmoid(),
    nn.Linear(8, 4),
    nn.Hardsigmoid(),
)

In [4]:
agent = Agent(device, model)

## Train Agent

In [5]:
num_games = int(1e3)
batch_size = int(1e6)
eps_start = 0.75
eps_end = 0.005
eps_decay = 75
discount = 0.999
loss_func = F.smooth_l1_loss
optimizer = optim.Adam(model.parameters())

In [6]:
session = agent.init_training_session(num_games, batch_size, eps_start, eps_end,
                                      eps_decay, discount, loss_func, optimizer)

In [7]:
record = pandas.DataFrame(numpy.zeros((num_games, 3)), columns=['Time (s)', 'Mean', 'Std'])

In [8]:
%%time
for index, game_time, mean, std in session:
    print(game_time, mean, std)
    record.loc[index] = [game_time, float(mean), float(std)]
record.round(2)

21.950833797454834 1103.9559326171875 539.6386108398438
43.64528775215149 1105.3509521484375 536.51171875
66.91377353668213 1103.6162109375 540.128662109375
88.81466674804688 1103.186279296875 538.4044799804688
110.92223763465881 1103.2974853515625 539.113525390625
132.05616879463196 1103.9927978515625 539.2705078125
153.18730759620667 1103.8055419921875 538.7001953125
175.29158520698547 1104.8677978515625 538.894287109375
198.20242619514465 1106.4688720703125 540.5900268554688
220.14101934432983 1106.47607421875 540.6373901367188
242.2395601272583 1105.8251953125 540.2598876953125
264.5092103481293 1107.1378173828125 541.2001953125
284.76872992515564 1108.564453125 541.209228515625
306.4050221443176 1109.1627197265625 541.0737915039062
327.5669484138489 1108.28076171875 540.381591796875
349.3932147026062 1109.85009765625 542.2599487304688
370.7385222911835 1110.6910400390625 542.1346435546875
392.111389875412 1110.185546875 542.546630859375
413.7042329311371 1112.0609130859375 543.654

KeyboardInterrupt: 

In [9]:
record.to_csv('records/trial2.csv', index=False)

In [10]:
torch.save(agent._model.state_dict(), 'model2.pt')